## Background

In this article, we will use [softmax](https://en.wikipedia.org/wiki/Softmax_function) classifier to build a simple image classification neural network with an accuracy of 32%. In a Softmax classifier, binary logic is generalized and regressed to multiple logic. Softmax classifier will output the probability of the corresponding category.

We will first define a softmax classifier, then use the training set of [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) to train the neural network, and finally use the test set to verify the accuracy of the neural network.

Let’s get started.

## Import dependencies

Like the previous course [GettingStarted](https://thoughtworksinc.github.io/DeepLearning.scala/demo/GettingStarted.html), we need to introduce each class of DeepLearning.scala.

In [0]:
import $ivy.`org.nd4j::nd4s:0.8.0`
import $ivy.`org.nd4j:nd4j-api:0.8.0`
import $ivy.`org.nd4j:nd4j-cuda-8.0-platform:0.8.0`
// import $ivy.`org.nd4j:nd4j-native-platform:0.8.0`
import $ivy.`com.chuusai::shapeless:2.3.2`
import $ivy.`org.rauschig:jarchivelib:0.5.0`
import $ivy.`com.thoughtworks.deeplearning::plugins-builtins:2.0.1`
import $ivy.`org.plotly-scala::plotly-jupyter-scala:0.3.2`
import $ivy.`com.thoughtworks.each::each:3.3.1`
import $plugin.$ivy.`org.scalamacros:paradise_2.11.11:2.1.0`

import scala.concurrent.ExecutionContext.Implicits.global
import org.nd4j.linalg.api.ndarray.INDArray
import org.nd4j.linalg.factory.Nd4j
import com.thoughtworks.deeplearning.DeepLearning
import com.thoughtworks.deeplearning.plugins._
import com.thoughtworks.feature.Factory
import plotly._
import plotly.element._
import plotly.layout._
import plotly.JupyterScala._
plotly.JupyterScala.init()

import com.thoughtworks.future._
import scala.concurrent.Await
import scala.concurrent.duration.Duration
import com.thoughtworks.each.Monadic._
import scalaz.std.stream._

: 

To reduce the line numbers outputted by `jupyter-scala` and to make sure that the page output will not be too long, we need to set `pprintConfig`.

In [1]:
pprintConfig() = pprintConfig().copy(height = 2)

## Build your own neural network.

### Set learning rate

Learning rate need to be set for the full connection layer. Learning rate visually describes the change rate of `weight`. A too-low learning rate will result in slow decrease of `loss`, which will require longer time for training; A too-high learning rate will result in rapid decrease of `loss` at first while fluctuation around the lowest point afterward.

In [2]:
val INDArrayLearningRatePluginUrl = "https://gist.githubusercontent.com/TerrorJack/118487016d7973d67feb489449dee156/raw/778bb1b68a664c752b0945111220326731310214/INDArrayLearningRate.sc"
interp.load(scala.io.Source.fromURL(new java.net.URL(INDArrayLearningRatePluginUrl)).mkString)

Downloaded https://repo1.maven.org/maven2/com/thoughtworks/deeplearning/plugins-weights_2.11/2.0.0-RC5/plugins-weights_2.11-2.0.0-RC5.pom
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/deeplearning/plugins-weights_2.11/2.0.0-RC5/plugins-weights_2.11-2.0.0-RC5.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/deeplearning/plugins-implicitssingleton_2.11/2.0.0-RC5/plugins-implicitssingleton_2.11-2.0.0-RC5.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/deeplearning/plugins-implicitssingleton_2.11/2.0.0-RC5/plugins-implicitssingleton_2.11-2.0.0-RC5.pom
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/deeplearning/plugins-weights_2.11/2.0.0-RC5/
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/deeplearning/plugins-implicitssingleton_2.11/2.0.0-RC5/
Downloaded https://repo1.maven.org/maven2/org/scalaz/scalaz-effect_2.11/7.2.11/scalaz-effect_2.11-7.2.11.pom
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/raii/as

Downloaded https://repo1.maven.org/maven2/com/thoughtworks/raii/covariant_2.11/1.0.1/covariant_2.11-1.0.1.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/raii/shared_2.11/1.0.1/shared_2.11-1.0.1.pom
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/raii/shared_2.11/1.0.1/shared_2.11-1.0.1.pom.sha1
Downloaded https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2.13/scalaz-core_2.11-7.2.13.pom.sha1
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/raii/covariant_2.11/1.0.1/
Downloaded https://repo1.maven.org/maven2/org/scalaz/scalaz-concurrent_2.11/7.2.11/
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/raii/shared_2.11/1.0.1/
Downloaded https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2.13/
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/deeplearning/plugins-implicitssingleton_2.11/2.0.0-RC5/plugins-implicitssingleton_2.11-2.0.0-RC5.jar
Downloaded https://repo1.maven.org/maven2/com/thoughtworks/tryt/covaria

https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2… (25.62 %, 30563…

https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2… (27.82 %, 33185…

https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2… (30.15 %, 35970…

https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2… (32.35 %, 38591…

https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2… (34.82 %, 41541…

https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2… (37.02 %, 44162…

https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2… (39.49 %, 47111…

https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2… (41.97 %, 50060…

https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2… (43.75 %, 52190…

https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2… (45.40 %, 54156…

https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2… (47.05 %, 56122…

https://repo1.maven.org/maven2/org/scalaz/scalaz-core_2.11/7.2… (48.70 %, 58088…

https://repo1.ma

Main1.sc:13: not found: type INDArray
    override def delta: INDArray = {
                        ^

: 

In [3]:
interp.load(scala.io.Source.fromURL(new java.net.URL("https://gist.github.com/Atry/15b7d9a4c63d95ad3d67e94bf20b4f69/raw/59f7ee4dff0dde3753f560633574265e950edc93/CNN.sc")).mkString)

Main1.sc:2: type INDArrayLayers is not a member of package com.thoughtworks.deeplearning.plugins
extends com.thoughtworks.deeplearning.plugins.INDArrayLayers
                                              ^Main1.sc:4: type Training is not a member of package com.thoughtworks.deeplearning.plugins
with com.thoughtworks.deeplearning.plugins.Training
                                           ^Main1.sc:5: type Operators is not a member of package com.thoughtworks.deeplearning.plugins
with com.thoughtworks.deeplearning.plugins.Operators {
                                           ^Main1.sc:15: object each is not a member of package com.thoughtworks
  import com.thoughtworks.each.Monadic._
                          ^Main1.sc:20: INDArrayLayers does not name a parent class of trait CNNs
    extends super[INDArrayLayers].ImplicitsApi
            ^Main1.sc:21: Training does not name a parent class of trait CNNs
      with super[Training].ImplicitsApi
           ^Main1.sc:22: Operators does not 

: 

In [4]:
// `interp.load` is a workaround for https://github.com/lihaoyi/Ammonite/issues/649 and https://github.com/scala/bug/issues/10390
interp.load("""
  val hyperparameters = Factory[Builtins with CNNs with INDArrayLearningRate].newInstance(learningRate = 0.1)
""")

Main1.sc:2: not found: value Factory
  val hyperparameters = Factory[Builtins with CNNs with INDArrayLearningRate].newInstance(learningRate = 0.1)
                        ^Main1.sc:2: not found: type Builtins
  val hyperparameters = Factory[Builtins with CNNs with INDArrayLearningRate].newInstance(learningRate = 0.1)
                                ^Main1.sc:2: not found: type CNNs
  val hyperparameters = Factory[Builtins with CNNs with INDArrayLearningRate].newInstance(learningRate = 0.1)
                                              ^Main1.sc:2: not found: type INDArrayLearningRate
  val hyperparameters = Factory[Builtins with CNNs with INDArrayLearningRate].newInstance(learningRate = 0.1)
                                                        ^Main1.sc:2: not found: value learningRate
  val hyperparameters = Factory[Builtins with CNNs with INDArrayLearningRate].newInstance(learningRate = 0.1)
                                                                                          

: 

### Write softmax

To use `softmax` classifier (softmax classifier is a neural network combined by `softmax` and a full connection), we first need to write softmax function, formula: ![](https://www.zhihu.com/equation?tex=f_j%28z%29%3D%5Cfrac%7Be%5E%7Bz_j%7D%7D%7B%5Csum_ke%5E%7Bz_k%7D%7D)

In [4]:
import hyperparameters.implicits._

cmd4.sc:1: not found: value hyperparameters
import hyperparameters.implicits._
       ^

: 

In [4]:
import hyperparameters.INDArrayLayer

def softmax(scores: INDArrayLayer): INDArrayLayer = {
  val expScores = hyperparameters.exp(scores)
  expScores / expScores.sum(1)
}

cmd4.sc:1: not found: value hyperparameters
import hyperparameters.INDArrayLayer
       ^cmd4.sc:3: not found: type INDArrayLayer
def softmax(scores: INDArrayLayer): INDArrayLayer = {
                                    ^cmd4.sc:3: not found: type INDArrayLayer
def softmax(scores: INDArrayLayer): INDArrayLayer = {
                    ^cmd4.sc:4: not found: value hyperparameters
  val expScores = hyperparameters.exp(scores)
                  ^

: 

In [4]:
val fileHandler = new java.util.logging.FileHandler("CNN%g.log")
hyperparameters.logger.addHandler(fileHandler)

cmd4.sc:2: not found: value hyperparameters
val res4_1 = hyperparameters.logger.addHandler(fileHandler)
             ^

: 

### Compose your  neural network

Define a full connection layer and [initialize Weight](https://github.com/ThoughtWorksInc/DeepLearning.scala/wiki/Getting-Started#231--weight-intialization), `Weight` shall be a two-dimension `INDArray` of `NumberOfPixels × NumberOfClasses`. `scores` is the score of each image corresponding to each category, representing the feasible probability of each category corresponding to each image.

In [5]:
//10 label of CIFAR10 images(airplane,automobile,bird,cat,deer,dog,frog,horse,ship,truck)
val NumberOfClasses: Int = 10
val NumberOfPixels: Int = 3072

NumberOfClasses: Int = 10
NumberOfPixels: Int = 3072

In [5]:
import hyperparameters.INDArrayWeight

val weight1 = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(Array(16, 3, 3, 3)) / math.sqrt(3 * 3 * 3 / 2))
}
val bias1 = {
    hyperparameters.INDArrayWeight(Nd4j.zeros(16))
}

val weight2 = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(Array(18, 16, 3, 3)) / math.sqrt(16 * 3 * 3 / 2))
}
val bias2 = {
    hyperparameters.INDArrayWeight(Nd4j.zeros(18))
}

val weight3 = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(Array(20, 18, 3, 3)) / math.sqrt(18 * 3 * 3 / 2))
}
val bias3 = {
    hyperparameters.INDArrayWeight(Nd4j.zeros(20))
}

val weight4 = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(Array(22, 20, 3, 3)) / math.sqrt(20 * 3 * 3 / 2))
}
val bias4 = {
    hyperparameters.INDArrayWeight(Nd4j.zeros(22))
}

val weight5 = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(Array(24, 22, 3, 3)) / math.sqrt(22 * 3 * 3 / 2))
}
val bias5 = {
    hyperparameters.INDArrayWeight(Nd4j.zeros(24))
}


val weight6 = {
    import org.nd4s.Implicits._
    INDArrayWeight(Nd4j.randn(Array(24, 10)) / math.sqrt(24 / 2))
}
val bias6 = {
    hyperparameters.INDArrayWeight(Nd4j.zeros(10))
}

def myNeuralNetwork(input: INDArray): INDArrayLayer = {
    import hyperparameters.max
    import hyperparameters.maxPool
    import hyperparameters.conv2d
    val layer1 = maxPool(max(conv2d(input.reshape(input.shape()(0), 3, 32, 32), weight1, bias1, (3, 3), (1, 1), (1, 1)), 0.0), (2, 2))
    val layer2 = maxPool(max(conv2d(layer1, weight2, bias2, (3, 3), (1, 1), (1, 1)), 0.0), (2, 2))
    val layer3 = maxPool(max(conv2d(layer2, weight3, bias3, (3, 3), (1, 1), (1, 1)), 0.0), (2, 2))
    val layer4 = maxPool(max(conv2d(layer3, weight4, bias4, (3, 3), (1, 1), (1, 1)), 0.0), (2, 2))
    val layer5 = maxPool(max(conv2d(layer4, weight5, bias5, (3, 3), (1, 1), (1, 1)), 0.0), (2, 2))

    val layer6 = layer5.reshape(input.shape()(0), 24) dot weight6 + bias6
    softmax(layer6)
}

cmd5.sc:1: not found: value hyperparameters
import hyperparameters.INDArrayWeight
       ^cmd5.sc:5: not found: value INDArrayWeight
    INDArrayWeight(Nd4j.randn(Array(16, 3, 3, 3)) / math.sqrt(3 * 3 * 3 / 2))
    ^cmd5.sc:5: not found: value Nd4j
    INDArrayWeight(Nd4j.randn(Array(16, 3, 3, 3)) / math.sqrt(3 * 3 * 3 / 2))
                   ^cmd5.sc:8: not found: value hyperparameters
    hyperparameters.INDArrayWeight(Nd4j.zeros(16))
    ^cmd5.sc:8: not found: value Nd4j
    hyperparameters.INDArrayWeight(Nd4j.zeros(16))
                                   ^cmd5.sc:13: not found: value INDArrayWeight
    INDArrayWeight(Nd4j.randn(Array(18, 16, 3, 3)) / math.sqrt(16 * 3 * 3 / 2))
    ^cmd5.sc:13: not found: value Nd4j
    INDArrayWeight(Nd4j.randn(Array(18, 16, 3, 3)) / math.sqrt(16 * 3 * 3 / 2))
                   ^cmd5.sc:16: not found: value hyperparameters
    hyperparameters.INDArrayWeight(Nd4j.zeros(18))
    ^cmd5.sc:16: not found: value Nd4j
    hyperparameters.INDArrayWeight(

: 

### Create LossFunction

To learn about the prediction result of the neural network, we need to write the loss function `lossFunction`. We use [cross-entropy loss](https://en.wikipedia.org/wiki/Cross_entropy) to make comparison between this result and the actual result before return the score. Formula:
![](https://zhihu.com/equation?tex=%5Cdisplaystyle+H%28p%2Cq%29%3D-%5Csum_xp%28x%29+logq%28x%29)

In [5]:
import hyperparameters.DoubleLayer

def lossFunction(input: INDArray, expectOutput: INDArray): DoubleLayer = {
    val probabilities = myNeuralNetwork(input)
    -(hyperparameters.log(probabilities) * expectOutput).mean
}

cmd5.sc:1: not found: value hyperparameters
import hyperparameters.DoubleLayer
       ^cmd5.sc:3: not found: type DoubleLayer
def lossFunction(input: INDArray, expectOutput: INDArray): DoubleLayer = {
                                                           ^cmd5.sc:3: not found: type INDArray
def lossFunction(input: INDArray, expectOutput: INDArray): DoubleLayer = {
                        ^cmd5.sc:3: not found: type INDArray
def lossFunction(input: INDArray, expectOutput: INDArray): DoubleLayer = {
                                                ^cmd5.sc:4: not found: value myNeuralNetwork
    val probabilities = myNeuralNetwork(input)
                        ^cmd5.sc:5: not found: value hyperparameters
    -(hyperparameters.log(probabilities) * expectOutput).mean
      ^

: 

## Prepare data

### Read data

To read the images and corresponding label information for test data from CIFAR10 database and process them, we need [`import $file.ReadCIFAR10ToNDArray`](https://github.com/ThoughtWorksInc/DeepLearning.scala-website/blob/master/ipynbs/ReadCIFAR10ToNDArray.sc). This is a script file containing the read and processed CIFAR10 data, provided in this course.

In [6]:
// import $url.{`https://raw.githubusercontent.com/ThoughtWorksInc/DeepLearning.scala-website/v1.0.0-doc/ipynbs/ReadCIFAR10ToNDArray.sc` => ReadCIFAR10ToNDArray}

// val trainNDArray = ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/data_batch_1.bin", 1000)

// val testNDArray = ReadCIFAR10ToNDArray.readFromResource("/cifar-10-batches-bin/test_batch.bin", 100)

import $ivy.`com.thoughtworks.deeplearning.etl::cifar10:1.0.1`
import com.thoughtworks.deeplearning.etl.Cifar10
import com.thoughtworks.future._
val cifar10 = Cifar10.load().blockingAwait

import $ivy.$                                                 

import com.thoughtworks.deeplearning.etl.Cifar10

import com.thoughtworks.future._

cifar10: com.thoughtworks.deeplearning.etl.Cifar10 = Cifar10(
  Vector(
...

### Process data

Before passing data to the softmax classifier, we first process label data with ([one hot encoding](https://en.wikipedia.org/wiki/One-hot)): transform INDArray of `NumberOfPixels × 1` into INDArray of `NumberOfPixels × NumberOfClasses`. The value of correct classification corresponding to each line is 1, and the values of other columns are 0. The reason for differentiating the training set and test set is to make it clear that whether the network is over trained which leads to [overfitting](https://en.wikipedia.org/wiki/Overfitting). While processing label data, we used [Utils](https://github.com/ThoughtWorksInc/DeepLearning.scala-website/blob/master/ipynbs/Utils.sc), which is also provided in this course.

In [6]:
// val trainData = trainNDArray.head
// val testData = testNDArray.head

// val trainExpectResult = trainNDArray.tail.head
// val testExpectResult = testNDArray.tail.head

// import $url.{`https://raw.githubusercontent.com/ThoughtWorksInc/DeepLearning.scala-website/v1.0.0-doc/ipynbs/Utils.sc` => Utils}

// val vectorizedTrainExpectResult = Utils.makeVectorized(trainExpectResult, NumberOfClasses)
// val vectorizedTestExpectResult = Utils.makeVectorized(testExpectResult, NumberOfClasses)

## Train your neural network

To observe the training process of the neural network, we need to output `loss`; while training the neural network, the `loss` shall be decreasing.

In [6]:
// var lossSeq: IndexedSeq[Double] = IndexedSeq.empty

// @monadic[Future]
// val trainTask: Future[Unit] = {
//   val lossStream = for (_ <- (1 to 2000).toStream) yield {
//     val loss = lossFunction(trainData, vectorizedTrainExpectResult).train.each
//     kernel.publish.markdown(s"loss: $loss")
//     loss
//   }
//   lossSeq = IndexedSeq.concat(lossStream)
// }

import scalaz.std.anyVal._
import scalaz.syntax.all._

class Trainer(batchSize: Int, numberOfEpoches: Int = 5) {
    @volatile
    private var isShuttingDown: Boolean = false

    private val lossBuffer = scala.collection.mutable.Buffer.empty[Float]
    
    def lossSeq: IndexedSeq[Float] = lossBuffer
    
    def poltLoss(): Unit = Seq(Scatter(lossSeq.indices, lossSeq)).plot(title = "loss by time")
    
    def interrupt(): Unit = isShuttingDown = true

    def startTrain(): Unit = {

        @monadic[Future]
        def trainTask: Future[Unit] = {
            isShuttingDown = false
            for (epoch <- 0 |=> numberOfEpoches) {
                val iterator = cifar10.epoch(batchSize).zipWithIndex
                while (iterator.hasNext && !isShuttingDown) {
                    val (Cifar10.Batch(labels, batch), i) = iterator.next()
                    val loss = lossFunction(batch, labels).train.each
                    lossBuffer += loss
                    hyperparameters.logger.info(s"epoch=$epoch iteration=$i batchSize=$batchSize loss=$loss")
                }
            }
            hyperparameters.logger.info("Done")
        }

        trainTask.onComplete { tryUnit: scala.util.Try[Unit] => tryUnit.get }

    }
}




cmd6.sc:10: type mismatch;
 found   : scala.collection.mutable.Buffer[Float]
 required: IndexedSeq[Float]
    def lossSeq: IndexedSeq[Float] = lossBuffer
                                     ^cmd6.sc:12: not found: value Scatter
    def poltLoss(): Unit = Seq(Scatter(lossSeq.indices, lossSeq)).plot(title = "loss by time")
                               ^cmd6.sc:12: not found: value title
    def poltLoss(): Unit = Seq(Scatter(lossSeq.indices, lossSeq)).plot(title = "loss by time")
                                                                       ^cmd6.sc:18: not found: type monadic
        @monadic[Future]
         ^cmd6.sc:21: value foreach is not a member of scalaz.EphemeralStream[Int]
            for (epoch <- 0 |=> numberOfEpoches) {
                            ^cmd6.sc:25: not found: value lossFunction
                    val loss = lossFunction(batch, labels).train.each
                               ^cmd6.sc:27: not found: value hyperparameters
                    hyperpara

: 

In [6]:
val trainer = new Trainer(batchSize = 32, numberOfEpoches = 300)
trainer.startTrain()

cmd6.sc:1: not found: type Trainer
val trainer = new Trainer(batchSize = 32, numberOfEpoches = 300)
                  ^cmd6.sc:1: not found: value batchSize
val trainer = new Trainer(batchSize = 32, numberOfEpoches = 300)
                          ^cmd6.sc:1: not found: value numberOfEpoches
val trainer = new Trainer(batchSize = 32, numberOfEpoches = 300)
                                          ^

: 

In [6]:
//
trainer.interrupt()

cmd6.sc:1: not found: value trainer
val res6 = trainer.interrupt()
           ^

: 

## Predict  your Neural Network

We use the processed test data to verify the prediction result of the neural network and compute the accuracy. The accuracy shall be about 32%.

In [6]:
val predictResult = Await.result(myNeuralNetwork(testData).predict.toScalaFuture, Duration.Inf)
println("The accuracy is " + Utils.getAccuracy(predictResult,testExpectResult) + "%")

cmd6.sc:1: not found: value Await
val predictResult = Await.result(myNeuralNetwork(testData).predict.toScalaFuture, Duration.Inf)
                    ^cmd6.sc:1: not found: value myNeuralNetwork
val predictResult = Await.result(myNeuralNetwork(testData).predict.toScalaFuture, Duration.Inf)
                                 ^cmd6.sc:1: not found: value testData
val predictResult = Await.result(myNeuralNetwork(testData).predict.toScalaFuture, Duration.Inf)
                                                 ^cmd6.sc:1: not found: value Duration
val predictResult = Await.result(myNeuralNetwork(testData).predict.toScalaFuture, Duration.Inf)
                                                                                  ^cmd6.sc:2: not found: value Utils
val res6_1 = println("The accuracy is " + Utils.getAccuracy(predictResult,testExpectResult) + "%")
                                          ^cmd6.sc:2: not found: value testExpectResult
val res6_1 = println("The accuracy is " + Utils.getAccu

: 

In [ ]:
trainer.poltLoss()

## Summary

We have learned the follows in this article:

* Prepare and process CIFAR10 data
* Write softmax classifier
* Use the prediction image of the neural network written by softmax classifier to match with the probability of each category.